In [4]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import pandas as pd


In [8]:
df  = pd.read_csv("data/Iris.csv")
df.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [10]:
df['Species'].value_counts()
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [130]:
X = df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']].values
Y = pd.get_dummies(df['Species']).values


In [131]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42
)

In [133]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, X, y):

        self.features = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype=torch.float32)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]        
        return x, y

    def __len__(self):
        return self.labels.shape[0]

In [140]:
train_ds = MyDataset(X_train, Y_train)
test_ds = MyDataset(X_test, Y_test)


(120, 4)
(120, 3)
torch.Size([120, 4])
torch.Size([120, 3])


# ML Model

In [141]:
class LogisticRegression(torch.nn.Module):
    def __init__(self,num_features,output_class):
        super().__init__()
        #3 is the no of output classes in Iris dataset
        self.linear = torch.nn.Linear(num_features,output_class)
    def forward(self, x):
        logits = self.linear(x)
        #probas = torch.sigmoid(logits)
        return logits

In [142]:
torch.manual_seed(1)
model = LogisticRegression(num_features=4,output_class=3)


# Define Dataloader

In [146]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=10,
    shuffle=True,
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=10,
    shuffle=True,
)


# Training a model

In [ ]:
import torch.nn.functional as F
import torch.nn as nn


torch.manual_seed(1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

num_epochs = 300
loss_func = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    
    model = model.train()
    for batch_idx, (features, class_labels) in enumerate(train_loader):

        probas = model(features)
        #print(probas.shape)
        #print(class_labels.shape)
        loss = loss_func(probas,class_labels.view(probas.shape))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### LOGGING
        print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}'
               f' | Batch {batch_idx:03d}/{len(train_loader):03d}'
               f' | Loss: {loss:.2f}')
            

# Testing a model

In [152]:
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for batch_idx, (features, class_labels) in enumerate(test_loader):
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        _, labels = torch.max(class_labels, 1)
        n_samples += class_labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {acc} %')

Accuracy: 100.0 %
